<a href="https://colab.research.google.com/github/BrixManzanero/CCADMACL_EXERCISES_COM222/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [5]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)
df_features = housing.data
df_target = housing.target

In [6]:
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

1. Use any filter method to select the best features

In [9]:
selector = VarianceThreshold(threshold=0.2) # Variance threshold
sel = selector.fit(X)
sel_index = sel.get_support()
df_wine_norm_vt = X.iloc[:, sel_index]
df_wine_norm_vt.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude'],
      dtype='object')

In [10]:
# put your answer here
variance_selector = VarianceThreshold(threshold=0.01)
X_train_variance = variance_selector.fit_transform(X_train_scaled)
X_test_variance = variance_selector.transform(X_test_scaled)

In [11]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_variance, y_train)

RandomForestRegressor(random_state=42)

In [12]:
y_pred_variance = rf.predict(X_test_variance)
rmse_variance = np.sqrt(mean_squared_error(y_test, y_pred_variance))
print(rmse_variance)

0.5051432839771741


2. Use any wrapper method to select the best features

In [13]:
# put your answer here
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

threshold = 5 # the number of most relevant features
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(X,y.values.ravel())
selector_ind = selector.get_support()
df_wine_rfe = X.iloc[:, selector_ind]
df_wine_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

In [17]:
y_pred_rfe = rf.predict(X_test_rfe)
rmse_rfe = np.sqrt(mean_squared_error(y_test, y_pred_rfe))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


ValueError: X has 5 features, but RandomForestRegressor is expecting 8 features as input.

3. Use any embedded methood to select the best features

In [193]:
# put your answer here
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)


Lasso(alpha=0.1)

In [1]:
from sklearn.feature_selection import SelectFromModel

model_rf = RandomForestClassifier(n_estimators=500, random_state=0, max_depth = 3)
model_rf.fit(df_wine_features, df_wine_target.values.ravel())

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_wine_sfm = df_wine_features.iloc[:, sel_sfm_index]
df_wine_sfm.columns

NameError: name 'RandomForestClassifier' is not defined

In [194]:
selected_features = np.where(lasso.coef_ != 0)[0]
X_train_lasso = X_train_scaled[:, selected_features]
X_test_lasso = X_test_scaled[:, selected_features]

In [196]:
y_pred_lasso = rf.predict(X_test_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

In [197]:
rf.fit(X_train_scaled, y_train)
y_pred_default = rf.predict(X_test_scaled)
rmse_default = np.sqrt(mean_squared_error(y_test, y_pred_default))

# Print the results
print(f"Default RMSE: {rmse_default}")
print(f"Filter (Variance) RMSE: {rmse_variance}")
print(f"Wrapper (RFE) RMSE: {rmse_rfe}")
print(f"Embedded (Lasso) RMSE: {rmse_lasso}")

Default RMSE: 0.5051432839771741
Filter (Variance) RMSE: 0.5051432839771741
Wrapper (RFE) RMSE: 0.5122974847844446
Embedded (Lasso) RMSE: 0.7245812624867553
